In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45649
2,Huelva,Confirmados PDIA 14 días,295
3,Huelva,Tasa PDIA 14 días,"57,485823411345166"
4,Huelva,Confirmados PDIA 7 días,137
5,Huelva,Tasa PDIA 7 dias,"26,696806126624704"
6,Huelva,Total Confirmados,45851
7,Huelva,Curados,44713
8,Huelva,Fallecidos,418


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45649.0


/tmp/ipykernel_22629/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12577.0


/tmp/ipykernel_22629/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_22629/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_22629/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_22629/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 11064 personas en los últimos 7 días 

Un positivo PCR cada 3425 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45649.0,137.0,295.0,513170.0,26.696806,57.485823,9.0
Condado-Campiña,14185.0,58.0,106.0,156231.0,37.124514,67.848250,4.0
Huelva-Costa,27011.0,66.0,160.0,289548.0,22.794148,55.258541,3.0
Moguer,1981.0,14.0,25.0,21867.0,64.023414,114.327525,2.0
Ayamonte,2088.0,5.0,18.0,21104.0,23.692191,85.291888,1.0
Cartaya,2027.0,27.0,56.0,20083.0,134.442065,278.842802,1.0
Palma del Condado (La),1422.0,6.0,6.0,10801.0,55.550412,55.550412,1.0
Palos de la Frontera,1042.0,11.0,13.0,11742.0,93.680804,110.713677,1.0
Punta Umbría,1273.0,7.0,10.0,15355.0,45.587756,65.125366,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Rosal de la Frontera,80.0,3.0,6.0,1697.0,176.782557,353.565115,0.0
Villarrasa,227.0,2.0,7.0,2211.0,90.456807,316.598824,0.0
Cartaya,2027.0,27.0,56.0,20083.0,134.442065,278.842802,1.0
Manzanilla,141.0,2.0,3.0,2118.0,94.428706,141.643059,0.0
Valverde del Camino,853.0,5.0,16.0,12750.0,39.215686,125.490196,0.0
Paterna del Campo,211.0,1.0,4.0,3457.0,28.926815,115.707261,NaN
Moguer,1981.0,14.0,25.0,21867.0,64.023414,114.327525,2.0
Palos de la Frontera,1042.0,11.0,13.0,11742.0,93.680804,110.713677,1.0
Bonares,464.0,4.0,6.0,6060.0,66.006601,99.009901,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Paterna del Campo,211.0,1.0,4.0,3457.0,28.926815,115.707261,0.0,0.250000
Ayamonte,2088.0,5.0,18.0,21104.0,23.692191,85.291888,1.0,0.277778
Villarrasa,227.0,2.0,7.0,2211.0,90.456807,316.598824,0.0,0.285714
Huelva (capital),12577.0,13.0,42.0,143837.0,9.038008,29.199719,0.0,0.309524
Valverde del Camino,853.0,5.0,16.0,12750.0,39.215686,125.490196,0.0,0.312500
Lepe,3025.0,6.0,18.0,27880.0,21.520803,64.562410,-1.0,0.333333
Almonte,2301.0,6.0,17.0,24507.0,24.482801,69.367936,-1.0,0.352941
Gibraleón,947.0,4.0,11.0,12737.0,31.404569,86.362566,0.0,0.363636
Bollullos Par del Condado,1289.0,3.0,8.0,14387.0,20.852158,55.605755,0.0,0.375000
